In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline

import warnings
warnings.simplefilter('ignore')

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
data = pd.read_csv('../input/train.tsv.zip',sep='\t')

In [ ]:
data.info()

In [ ]:
data.head()

 The sentiment labels are:

0 - negative; 
1 - somewhat negative; 
2 - neutral; 
3 - somewhat positive; 
4 - positive

In [ ]:
print(data.iloc[0]['Phrase'],'Sentiment - ',data.iloc[0]['Sentiment'])

The part of the sentence above which says 'but none of which amounts to much of a story' corresponds to a neagtive sentiment which is correctly indicated in the sentiment label '1'. Hence if we remove this phrase from the sentence as is done below, we get a neutral sentiment as the label!

In [ ]:
print(data.iloc[1]['Phrase'],'Sentiment - ',data.iloc[1]['Sentiment'])

Addition of a word like 'for' has shifted the sentiment from negative to neutral as shown below. Hence it is not recommended to use stopwords filtering here as we are not analysing full messages here, but phrases from the same sentence. A stopword like 'but' or 'not' can really alter the sentiment and hence filtering out them will be counterproductive.

In [ ]:
print(data.iloc[32]['Phrase'],'Sentiment - ',data.iloc[32]['Sentiment'])
print('\n')
print(data.iloc[33]['Phrase'],'Sentiment - ',data.iloc[33]['Sentiment'])

Lets clean the phrases by removing punctuation marks and splitting them into a list

In [ ]:
import string
string.punctuation

In [ ]:
from nltk.stem import WordNetLemmatizer
lm = WordNetLemmatizer()

def own_analyser(phrase):
    phrase = phrase.split()
    for i in range(0,len(phrase)):
        k = phrase.pop(0)
        if k not in string.punctuation:
                phrase.append(lm.lemmatize(k).lower())    
    return phrase

In [ ]:
data.columns

In [ ]:
X = data['Phrase']
y = data['Sentiment']

In [ ]:
from sklearn.model_selection import train_test_split
phrase_train,phrase_test,sentiment_train,sentiment_test = train_test_split(X,y,test_size=0.3)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB


Using pipeline feature of sklearn - 

In [ ]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([('BOW',CountVectorizer(analyzer=own_analyser)),
                    ('tfidf',TfidfTransformer()),
                    ('classifier',MultinomialNB())])

In [ ]:
pipeline.fit(phrase_train,sentiment_train)

In [ ]:
predictions = pipeline.predict(phrase_test)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(sentiment_test,predictions))

In [ ]:
test_data = pd.read_csv('../input/test.tsv.zip',sep='\t')

In [ ]:
test_data.head()

In [ ]:
test_predictions = pipeline.predict(test_data['Phrase'])

In [ ]:
phrase_id = test_data['PhraseId'].values

In [ ]:
test_predictions.shape

In [ ]:
final_answer = pd.DataFrame({'PhraseId':phrase_id,'Sentiment':test_predictions})

In [ ]:
final_answer.head()

In [ ]:
filename = 'Sentiment Analysis - NaiveBayes.csv'

final_answer.to_csv(filename,index=False)

print('Saved file: ' + filename)